# 🤖 Agente Pokémon — ReAct + PokéAPI + RAG + Memoria (LangGraph)
Incluye tools de comparación con esquema de argumentos (`compare_pokemon` + `compare_stat`).
Ejecuta cada celda de arriba hacia abajo. Requiere `OPENAI_API_KEY` en tu `.env`.


In [8]:
from dotenv import load_dotenv
load_dotenv()
from langchain_openai import ChatOpenAI
import os

assert os.getenv('OPENAI_API_KEY'), '⚠️ Falta OPENAI_API_KEY en el entorno o archivo .env'

llm = ChatOpenAI(
    model='gpt-4o-mini',
    temperature=0.1,
    max_tokens=600,
    timeout=30,
    max_retries=2
)
print('✅ LLM configurado correctamente')

✅ LLM configurado correctamente


## 📚 RAG: Embeddings + Chroma (conocimiento/lore de Pokémon)
Reemplaza `seed_docs` por tus propios textos cuando quieras.

In [9]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_core.documents import Document
import os

CHROMA_DIR = 'chroma_pokemon_knowledge'
os.makedirs(CHROMA_DIR, exist_ok=True)

seed_docs = [
    Document(page_content='Gengar se oculta en las sombras y absorbe el calor del entorno.', metadata={'source':'lore'}),
    Document(page_content='Alakazam posee un CI muy alto y recuerda todo desde su nacimiento.', metadata={'source':'lore'}),
    Document(page_content='Pikachu almacena electricidad en sus mejillas y puede liberar potentes descargas.', metadata={'source':'lore'}),
    Document(page_content='Charizard puede derretir rocas con su aliento de fuego.', metadata={'source':'lore'}),
    Document(page_content='Bulbasaur realiza fotosíntesis con la semilla de su espalda.', metadata={'source':'lore'})
]

emb = OpenAIEmbeddings()
vectorstore = Chroma.from_documents(seed_docs, embedding=emb, persist_directory=CHROMA_DIR)
retriever = vectorstore.as_retriever(search_kwargs={'k': 3})
print('✅ RAG (Chroma) listo')

✅ RAG (Chroma) listo


## 🔧 Tools: PokéAPI + Tool de RAG + Comparadores con schema

In [10]:
from langchain_core.tools import tool
from langchain_core.pydantic_v1 import BaseModel, Field
import requests

BASE = 'https://pokeapi.co/api/v2'

def _norm(s: str) -> str:
    return s.strip().lower().replace(' ', '-').replace('.', '').replace("'", '')

# 1) Información de un Pokémon
@tool('get_pokemon_info')
def get_pokemon_info(name: str):
    """Devuelve tipos, stats, altura y peso de un Pokémon por nombre."""
    r = requests.get(f"{BASE}/pokemon/{_norm(name)}", timeout=20)
    r.raise_for_status()
    d = r.json()
    return {
        'name': d['name'],
        'id': d.get('id'),
        'types': [t['type']['name'] for t in d['types']],
        'stats': {s['stat']['name']: s['base_stat'] for s in d['stats']},
        'height': d.get('height'),
        'weight': d.get('weight')
    }

# 2) Información de un tipo
@tool('get_type_info')
def get_type_info(type_name: str):
    """Devuelve fortalezas y debilidades de un tipo Pokémon."""
    r = requests.get(f"{BASE}/type/{_norm(type_name)}", timeout=20)
    r.raise_for_status()
    d = r.json()
    rel = d['damage_relations']
    return {
        'name': d['name'],
        'double_damage_to':   [t['name'] for t in rel['double_damage_to']],
        'double_damage_from': [t['name'] for t in rel['double_damage_from']],
        'half_damage_to':     [t['name'] for t in rel['half_damage_to']],
        'half_damage_from':   [t['name'] for t in rel['half_damage_from']],
        'no_damage_to':       [t['name'] for t in rel['no_damage_to']],
        'no_damage_from':     [t['name'] for t in rel['no_damage_from']]
    }

# 3) Información de un movimiento
@tool('get_move_info')
def get_move_info(move_name: str):
    """Devuelve precisión, poder, PP, tipo y clase de daño de un movimiento."""
    r = requests.get(f"{BASE}/move/{_norm(move_name)}", timeout=20)
    r.raise_for_status()
    d = r.json()
    return {
        'name': d['name'],
        'accuracy': d.get('accuracy'),
        'power': d.get('power'),
        'pp': d.get('pp'),
        'type': d['type']['name'],
        'damage_class': d['damage_class']['name']
    }

# 4) Caminos evolutivos
def _walk_chain(chain):
    paths = []
    def dfs(node, path):
        curr = node['species']['name']
        new_path = path + [curr]
        if not node['evolves_to']:
            paths.append(new_path)
        else:
            for nxt in node['evolves_to']:
                dfs(nxt, new_path)
    dfs(chain, [])
    return paths

@tool('get_evolution_paths')
def get_evolution_paths(pokemon_name: str):
    """Devuelve todas las rutas evolutivas posibles de un Pokémon."""
    r = requests.get(f"{BASE}/pokemon-species/{_norm(pokemon_name)}", timeout=20)
    r.raise_for_status()
    species = r.json()
    evo_url = species['evolution_chain']['url']
    r = requests.get(evo_url, timeout=20)
    r.raise_for_status()
    chain = r.json()['chain']
    return _walk_chain(chain)

# 5) Comparación de TODOS los stats (con schema)
class CompareArgs(BaseModel):
    """Compara los stats base de dos Pokémon."""
    pokemon1: str = Field(..., description="Nombre del primer Pokémon. Ej: 'Gengar'")
    pokemon2: str = Field(..., description="Nombre del segundo Pokémon. Ej: 'Alakazam'")

@tool('compare_pokemon', args_schema=CompareArgs)
def compare_pokemon(pokemon1: str, pokemon2: str):
    """Compara los stats base de dos Pokémon y devuelve el ganador por cada stat."""
    def _get_stats(name):
        r = requests.get(f"{BASE}/pokemon/{_norm(name)}", timeout=20)
        r.raise_for_status()
        d = r.json()
        return {s['stat']['name']: s['base_stat'] for s in d['stats']}
    stats1 = _get_stats(pokemon1)
    stats2 = _get_stats(pokemon2)
    result = {}
    for stat in stats1:
        if stats1[stat] > stats2[stat]:
            winner = pokemon1
        elif stats1[stat] < stats2[stat]:
            winner = pokemon2
        else:
            winner = 'Empate'
        result[stat] = {pokemon1: stats1[stat], pokemon2: stats2[stat], 'winner': winner}
    return result

# 6) Comparación de UN solo stat (con schema)
class CompareStatArgs(BaseModel):
    """Compara un stat específico entre dos Pokémon."""
    pokemon1: str = Field(..., description="Nombre del primer Pokémon")
    pokemon2: str = Field(..., description="Nombre del segundo Pokémon")
    stat: str = Field(..., description="Stat a comparar. Ej: 'speed', 'attack', 'defense', 'special-attack', 'special-defense', 'hp'")

@tool('compare_stat', args_schema=CompareStatArgs)
def compare_stat(pokemon1: str, pokemon2: str, stat: str):
    """Compara un único stat entre dos Pokémon y devuelve el ganador."""
    def _get_stats(name):
        r = requests.get(f"{BASE}/pokemon/{_norm(name)}", timeout=20)
        r.raise_for_status()
        d = r.json()
        return {s['stat']['name']: s['base_stat'] for s in d['stats']}
    stats1 = _get_stats(pokemon1)
    stats2 = _get_stats(pokemon2)
    val1, val2 = stats1.get(stat), stats2.get(stat)
    if val1 is None or val2 is None:
        return {'error': f"Stat '{stat}' no encontrado. Usa uno de: {list(stats1.keys())}"}
    if val1 > val2:
        winner = pokemon1
    elif val2 > val1:
        winner = pokemon2
    else:
        winner = 'Empate'
    return {'stat': stat, pokemon1: val1, pokemon2: val2, 'winner': winner}

# 7) 🔎 Tool RAG
@tool('search_pokemon_knowledge')
def search_pokemon_knowledge(query: str):
    """Busca en el vector store (lore/trivia) y devuelve fragmentos relevantes."""
    docs = retriever.invoke(query)
    return [{'content': d.page_content, 'source': d.metadata.get('source', 'unknown')} for d in docs]

# Lista final de tools
TOOLS = [
    get_pokemon_info,
    get_type_info,
    get_move_info,
    get_evolution_paths,
    compare_pokemon,
    compare_stat,
    search_pokemon_knowledge
]
print('✅ Tools Pokémon + RAG listos (con schemas de comparación)')

✅ Tools Pokémon + RAG listos (con schemas de comparación)


C:\Users\PC\AppData\Roaming\Python\Python312\site-packages\IPython\core\interactiveshell.py:3699: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


## 🧠 Agente ReAct (español) usando tools y RAG

In [11]:
from langchain import hub
from langchain.agents import create_react_agent, AgentExecutor

prompt = hub.pull('hwchase17/react')
extra_instruction = '''\nResponde SIEMPRE en español.\n- Para **datos técnicos** (stats, tipos, evoluciones, movimientos), usa las tools de **PokéAPI**.\n- Para **lore/trivia**, usa **search_pokemon_knowledge** (RAG) y cita brevemente la fuente entre [corchetes].\n- Para comparar TODOS los stats usa `compare_pokemon` (requiere pokemon1 y pokemon2).\n- Para comparar un solo stat usa `compare_stat` con el nombre del stat.\n- Si usas `compare_pokemon`, muestra una **tabla Markdown**: Stat | Valor de Pokémon 1 | Valor de Pokémon 2 | Ganador; y añade un **resumen** con el ganador global.\n'''
prompt = prompt.partial(instructions=extra_instruction)

agent = create_react_agent(llm, TOOLS, prompt)
agent_executor = AgentExecutor(agent=agent, tools=TOOLS, verbose=True)
print('✅ AgentExecutor listo (con RAG y comparadores)')

✅ AgentExecutor listo (con RAG y comparadores)


## 🧵 Memoria con LangGraph (thread_id) y ejemplo

In [12]:
from langgraph.graph import StateGraph, END
from langgraph.graph.message import MessagesState
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import AIMessage

def agent_node(state: MessagesState):
    msgs = state['messages']
    last_user = None
    for m in reversed(msgs):
        if isinstance(m, tuple):
            role, content = m
            if role == 'user':
                last_user = content
                break
        else:
            if getattr(m, 'type', '') == 'human':
                last_user = m.content
                break
    if not last_user:
        last_user = 'Hola, ¿en qué puedo ayudarte con Pokémon?'

    result = agent_executor.invoke({'input': last_user})
    answer = result.get('output', '')
    return {'messages': [AIMessage(content=answer)]}

graph = StateGraph(MessagesState)
graph.add_node('agent', agent_node)
graph.set_entry_point('agent')
graph.add_edge('agent', END)

memory = MemorySaver()
app = graph.compile(checkpointer=memory)
print('✅ app creada con memoria (LangGraph)')

✅ app creada con memoria (LangGraph)


In [16]:
# ▶️ Ejemplos con hilo (memoria)
config = {'configurable': {'thread_id': 'pokemon-demo-1'}}

# 1) Pregunta general + comparación puntual de VELOCIDAD
r1 = app.invoke({'messages': [('user', '¿De qué tipo es Gengar y contra qué tipos es fuerte o débil?')]}, config)
print(r1['messages'][-1].content)

# 2) Comparación de TODOS los stats
r2 = app.invoke({'messages': [('user', 'Dime sobre su historia ')]}, config)
print(r2['messages'][-1].content)



> Entering new AgentExecutor chain...
Para responder a la pregunta, primero necesito obtener información sobre Gengar, específicamente su tipo. Luego, puedo averiguar contra qué tipos es fuerte o débil. 

Action: get_pokemon_info  
Action Input: "Gengar"  {'name': 'gengar', 'id': 94, 'types': ['ghost', 'poison'], 'stats': {'hp': 60, 'attack': 65, 'defense': 60, 'special-attack': 130, 'special-defense': 75, 'speed': 110}, 'height': 15, 'weight': 405}Ahora sé que Gengar es de tipo Fantasma y Veneno. A continuación, necesito averiguar contra qué tipos es fuerte o débil.

Action: get_type_info  
Action Input: "ghost"  {'name': 'ghost', 'double_damage_to': ['ghost', 'psychic'], 'double_damage_from': ['ghost', 'dark'], 'half_damage_to': ['dark'], 'half_damage_from': ['poison', 'bug'], 'no_damage_to': ['normal'], 'no_damage_from': ['normal', 'fighting']}Ahora tengo información sobre el tipo Fantasma. Gengar es fuerte contra los tipos Fantasma y Psíquico, y débil contra los tipos Fantasma y 